In [2]:
import pandas 
import csv
import random
import copy

#Setting population size
populationSize = 100


################################################### DATA STRUCTURES #########################################################
class Teacher:
    def __init__(self,i,n):
        self.id = i
        self.name = n

class Student:
    def __init__(self,i,n):
        self.id = i
        self.name = n

class Course:
    def __init__(self,i,c,n):
        self.id = i
        self.code = c
        self.name = n

class Registration:
    def __init__(self,i,sn,cc):
        self.id = i
        self.studentName = sn
        self.courseCode  = cc
class studetcourse:
    def __init__(self,i):
        self.id = i
        self.courses=[]
class Exam:
    def __init__(self,time,r,day,subj,teacher):
        self.Time = time
        self.Day = day
        self.Room = r 
        self.Subj = subj
        self.Teacher  = teacher
        self.Gene = ""   




class TimeTable:
    def __init__(self,f):
        self.Exams = [] 
        self.fitness = f
        self.chromosome = ""

##################################################  FUNCTIONS ########################################################        


def printExam(i):
    print("Slot: ", i.Time)
    print("DAY: ",i.Day)
    print("Room: ",i.Room)
    print("Subj: ",i.Subj)
    print("Teacher: ",i.Teacher)
    print("Gene: ",i.Gene)


def printTimeTable(table):
    count=0
    for i in table.Exams:
        print("exam ", count+1)
        print("Slot: ", time_slot[i.Time])
        print("DAY: ",days[i.Day])
        print("Room: ",rooms[i.Room])
        print("Subj: ",courses[i.Subj].code, courses[i.Subj].name)
        print("Teacher: ",teachers[i.Teacher].name)
        print("Gene: ",i.Gene)
        print("")
        count+=1    

def toBinary(x):
    binary = ("{0:b}".format(x))
    return (binary) 

def toInteger(b):
    x = int(b, 2)
    return(x)   


def setGenes(ex):
    ex.Gene = toBinary(ex.Time)

    if(len(toBinary(ex.Day))<4):
        k=0
        while(k+len(toBinary(ex.Day))<4):
            ex.Gene +="0"
            k+=1
    ex.Gene += toBinary(ex.Day)
    
    if(len(toBinary(ex.Room))<4):
        k=0
        while(k+len(toBinary(ex.Room))<4):
            ex.Gene +="0"
            k+=1
    ex.Gene += toBinary(ex.Room)
    
    if(len(toBinary(ex.Subj))<5):
        k=0
        while(k+len(toBinary(ex.Subj))<5):
            ex.Gene +="0"
            k+=1
    ex.Gene += toBinary(ex.Subj)

    if(len(toBinary(ex.Teacher))<6):
        k=0
        while(k+len(toBinary(ex.Teacher))<6):
            ex.Gene +="0"
            k+=1
    ex.Gene += toBinary(ex.Teacher)

def correctValueAccordingToBinary(tble):
    for i in tble.Exams:
        i.Time = toInteger(i.Gene[0])
        i.Day = toInteger(i.Gene[1] + i.Gene[2] + i.Gene[3] + i.Gene[4])
        i.Room = toInteger(i.Gene[5] + i.Gene[6] + i.Gene[7] + i.Gene[8])
        i.Subj = toInteger(i.Gene[9] + i.Gene[10] + i.Gene[11] + i.Gene[12] + i.Gene[13])
        i.Teacher = toInteger(i.Gene[14] + i.Gene[15] + i.Gene[16] + i.Gene[17] + i.Gene[18] + i.Gene[19])
        #print(i.Day)

def calculateFitness(tble,students,courses,reg,teachers,sc,cs):
    temp = 0 

    
    for i in range(25):
        j=i+1
        while(j<26):
            #Exam repeat (also means some exam is not scheduled)
            if(tble.Exams[i].Subj==tble.Exams[j].Subj):
                temp+=1

            #Student cannot give more than 1 exam at a time
            if(tble.Exams[i].Day==tble.Exams[j].Day and  tble.Exams[i].Time==tble.Exams[j].Time):
                for itera in sc.keys():
                    if tble.Exams[i].Subj and tble.Exams[j].Subj in sc[itera]:
                        temp += 1
            
            #same teacher invigilating 2 exams at a time
            if(tble.Exams[i].Day==tble.Exams[j].Day and  tble.Exams[i].Time==tble.Exams[j].Time and  tble.Exams[i].Teacher==tble.Exams[j].Teacher):
                temp+=1

            #Same invigilator in a day
            if(tble.Exams[i].Day==tble.Exams[j].Day and  tble.Exams[i].Teacher==tble.Exams[j].Teacher):
                temp+=1

            #same say, same room, same time no 2 exams can happen
            if(tble.Exams[i].Day==tble.Exams[j].Day and  tble.Exams[i].Time==tble.Exams[j].Time and tble.Exams[i].Room==tble.Exams[j].Room):
                temp+=1

            #A student shall not give more than 1 exam consecutively.
            if(tble.Exams[i].Day==tble.Exams[j].Day):
                for itera in sc.keys():
                    if tble.Exams[i].Subj and tble.Exams[j].Subj in sc[itera]:
                        temp += 1
            
            j+=1

    #Student enrolled in atleast 3 courses
    for i in sc.keys():
        if len(sc[i]) < 3:
            temp += 1

    #Out of bonds
    for i in tble.Exams:
        if(i.Day > 12):
            temp+=1000
        if(i.Room > 9):
            temp+=1000
        if(i.Subj > 25):
            temp+=1000
        if(i.Teacher > 62):
            temp+=1000            

    tble.fitness = temp

def bubbleSort(arr):
    n = len(arr)
  
    # Traverse through all array elements
    for i in range(n-1):
    # range(n) also work but outer loop will repeat one time more than needed.
  
        # Last i elements are already in place
        for j in range(0, n-i-1):
  
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if arr[j].fitness > arr[j+1].fitness:
                arr[j], arr[j+1] = arr[j+1], arr[j]
  

def setChromosome(population):
    for i in population:
        j=0
        for j in range(26):
            i.chromosome +=i.Exams[j].Gene
             

def setGeneFromChromosome(tble):
    c=0
    for i in tble.Exams:
        i.Gene=""
        for j in range(20):
            i.Gene += tble.chromosome[c]
            c+=1
        correctValueAccordingToBinary(tble)    

def selectionftn (population):
    return population[0],population[1]

def singlePointCrossover (a,b):
    dummy1 = copy.deepcopy(a)
    dummy2 = copy.deepcopy(b)
    p = random.randint(1,519)
    while (p%20 != 0):
        p = random.randint(1,519)
    t1= dummy1.chromosome[0:p] + dummy2.chromosome[p:]
    t2=dummy2.chromosome[0:p] + dummy1.chromosome[p:]
    dummy1.chromosome = copy.deepcopy(t1)
    dummy2.chromosome = copy.deepcopy(t2)
    return dummy1, dummy2

def randomCrossover(a,b):
    dummy1 = copy.deepcopy(a)
    dummy2 = copy.deepcopy(b)
    p = random.randint(1,519)
    dummy1.chromosome[0:p] + dummy2.chromosome[p:]
    dummy2.chromosome[0:p] + dummy1.chromosome[p:]
    return dummy1, dummy2

def randomEffectiveCrossover(a,b):
    pH1= copy.deepcopy(a)
    pH2= copy.deepcopy(b)
    dummy1 = ""
    dummy2 = ""
    for i in range (10):
        p = random.randint(1,519)
        while (p%20 != 0):
            p = random.randint(1,519)
        dummy1 += a[0:p] + b[p:]
        dummy2 += b[0:p] + a[p:]
    pH1.chromosome = dummy1
    pH2.chromosome = dummy2
    return pH1,pH2



def corssover(population):
    
    for i in  range (len(population)):
        t1,t2 = singlePointCrossover(population[i],population[random.randint(0,populationSize-1)])
        population.append(t1)
        population.append(t2)
        i+=1

def bitflip(a,tble):
    if(tble.chromosome[a] =='0'):
        tble.chromosome = tble.chromosome[0:a] + "1" + tble.chromosome[a+1:]   
    else:
        tble.chromosome = tble.chromosome[0:a] + "0" + tble.chromosome[a+1:]

def mutation(population,students,courses,reg,teachers,studentC):

    for i in range (len(population)):
        if(i==0):
            continue
        bitflip(random.randint(0,519),population[i])
        bitflip(random.randint(0,519),population[i])       

################################################## Reading and initializing from CSV #####################################
c=1
teachers = []
with open('teachers.csv') as t_file:
    csv_reader = csv.reader(t_file)
    line_count = 0
    for row in csv_reader:
        if(str(row)!="[]"):
            teachers.append(Teacher(c,str(row)))
            c+=1    
count=0    
for i in teachers:
        teachers[count].name = teachers[count].name[2:len(teachers[count].name)-2]
        count+=1

c=1
students = []
with open('studentNames.csv') as sn_file:
    csv_reader1 = csv.reader(sn_file)
    line_count1 = 0
    for row1 in csv_reader1:
        if(str(row1)!="[]"):
            students.append(Student(c,str(row1)))
            c+=1    
count1=0    
for i in students:
        students[count1].name = students[count1].name[2:len(students[count1].name)-2]               
        count1+=1        


c=1
courses = []
with open('courses.csv') as c_file:
    csv_reader2 = csv.reader(c_file)
    line_count2 = 0
    for row2 in csv_reader2:
        if(str(row2)!="[]"):
            courses.append(Course(c,str(row2[0]),str(row2[1])))
            c+=1    

c=0
reg = []
temp1 = []
temp2= []
with open('studentCourse.csv') as cs_file:
    csv_reader3 = csv.reader(cs_file)
    line_count3 = 0
    for row3 in csv_reader3:
        if(str(row3)!="[]" ):
            if str(row3[1]) != 'Student Name':
                reg.append(Registration(c,str(row3[1]),str(row3[2])))
                temp1.append(str(row3[1]))
                temp2.append(str(row3[2]))
                c+=1

studentCourse = {}
courseStudent = {}
for i in range(len(reg)):
    if temp1[i] not in studentCourse.keys():
        studentCourse[temp1[i]] = []
        studentCourse[temp1[i]].append(temp2[i])
    else:
        studentCourse[temp1[i]].append(temp2[i])

for i in range(len(reg)):
    if temp2[i] not in courseStudent.keys():
        courseStudent[temp2[i]] = []
        courseStudent[temp2[i]].append(temp1[i])
    else:
        courseStudent[temp2[i]].append(temp1[i])

studentC = []
i=0
for i in range(200):
    studentC.append(studetcourse(i))
    j=0
    for j in range(820):
        #cnt=0
        if(reg[j].studentName == students[i].name):
            for z in range(26):
                if(reg[j].courseCode == courses[z].code):
                    icode = z
                    studentC[i].courses.append(icode)
                    break


days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Monday2","Tuesday2","Wednesday2","Thursday2","Friday2","Monday3","Tuesday3","Wednesday3","Thursday3","Friday3"]
rooms = [1,2,3,4,5,6,7,8,9,10]
time_slot = ["9-12","2-5"]


##########################  CREATING POPULATION #################################################################
population = []
for i in range(populationSize):
    #print(i)
    timebl=  TimeTable(0)
    for j in range (26):
        timebl.Exams.append(Exam(random.randint(0,1),random.randint(0,12),random.randint(0,9),random.randint(0,25),random.randint(0,62)))
    population.append(timebl)
    
#setting genes    
for i in range(populationSize):
    for j in range(26):
        setGenes(population[i].Exams[j])    
        

setChromosome(population)

#correcting
for i in range(populationSize):
    correctValueAccordingToBinary(population[i])   
#calling fitness
for i in range(populationSize):
    calculateFitness(population[i],students,courses,reg,teachers,studentCourse,courseStudent)

bubbleSort(population)

m=0

while(m<250):
    print("\nCurrent generation: ",m, "\nFittest individual have fitness:" ,population[0].fitness,"\t(lower is better!)")
    for i in population:
        if(i.fitness == 0):      
            break
        
    corssover(population)

    bubbleSort(population)
    if(population[0].fitness == 0):
            break

    c=(populationSize*3)-1
    for i in range(populationSize * 2):
        population.pop(c)
        c-=1

    mutation(population,students,courses,reg,teachers,studentC)

    for i in population:
        setGeneFromChromosome(i)

    for i in range(populationSize):
        calculateFitness(population[i],students,courses,reg,teachers,studentCourse,courseStudent)
    
    bubbleSort(population)
    
    m+=1         


if(population[0].fitness == 0):
    print("solution found!!!!")
    printTimeTable(population[0])


else:    
    print("run again")



Current generation:  0 
Fittest individual have fitness: 1010 	(lower is better!)

Current generation:  1 
Fittest individual have fitness: 1010 	(lower is better!)

Current generation:  2 
Fittest individual have fitness: 1010 	(lower is better!)

Current generation:  3 
Fittest individual have fitness: 1010 	(lower is better!)

Current generation:  4 
Fittest individual have fitness: 1010 	(lower is better!)

Current generation:  5 
Fittest individual have fitness: 10 	(lower is better!)

Current generation:  6 
Fittest individual have fitness: 10 	(lower is better!)

Current generation:  7 
Fittest individual have fitness: 10 	(lower is better!)

Current generation:  8 
Fittest individual have fitness: 9 	(lower is better!)

Current generation:  9 
Fittest individual have fitness: 9 	(lower is better!)

Current generation:  10 
Fittest individual have fitness: 8 	(lower is better!)

Current generation:  11 
Fittest individual have fitness: 8 	(lower is better!)

Current generation:


Current generation:  102 
Fittest individual have fitness: 1 	(lower is better!)

Current generation:  103 
Fittest individual have fitness: 1 	(lower is better!)

Current generation:  104 
Fittest individual have fitness: 1 	(lower is better!)

Current generation:  105 
Fittest individual have fitness: 1 	(lower is better!)

Current generation:  106 
Fittest individual have fitness: 0 	(lower is better!)
solution found!!!!
exam  1
Slot:  9-12
DAY:  Friday
Room:  4
Subj:  EE229 Computer Organization & Assembly Language
Teacher:  Farah Naz
Gene:  00100001110000000010

exam  2
Slot:  9-12
DAY:  Friday2
Room:  7
Subj:  MG220 Marketing Management
Teacher:  Shams Farooq
Gene:  01001011010100100001

exam  3
Slot:  2-5
DAY:  Tuesday
Room:  2
Subj:  EE227 Digital Logic Design
Teacher:  Tayyab Nadeem
Gene:  10001000100001001010

exam  4
Slot:  9-12
DAY:  Wednesday
Room:  9
Subj:  CS328 Software Engineering
Teacher:  Mehwish Hassan
Gene:  00010100001010100000

exam  5
Slot:  2-5
DAY:  Monday
Ro